In [2]:
from pathlib import Path
from pprint import pprint
import numpy as np
import pandas as pd

data_path = Path('.').resolve().parents[0].joinpath("data")
raw_path = data_path.joinpath("raw")
interim_path = data_path.joinpath("interim")

In [4]:
def show_nan(df, detail=True):
    print('missing value columns:')
    print(df.isnull().sum())

    if detail:
        nan_rows = df[df.isnull().any(axis=1)]
        if nan_rows.size > 0:
            print(nan_rows.head())

def mem_usage(pd_obj):
    if isinstance(pd_obj, pd.DataFrame):
        usage_bytes = pd_obj.memory_usage(deep=True).sum()
    else:
        usage_bytes = pd_obj.memory_usage(deep=True)
    usage_mb = usage_bytes / 1024 ** 2
    return "{:03.2f} MB".format(usage_mb)


def compare_mem_usage(b, a):
    print('{0} -> {1}'.format(mem_usage(action_int), mem_usage(converted_int)))

    compare = pd.concat([b.dtypes, a.dtypes], axis=1)
    compare.columns = ['before', 'after']
    print(compare.apply(pd.Series.value_counts))

    
def get_column_types(dtypes):
    dtypes_col = dtypes.index
    dtypes_type = [i.name for i in dtypes.values]
    return dict(zip(dtypes_col, dtypes_type))


def show_dtypes(df):
    dt_dtpyes = df.select_dtypes(include=[np.datetime64, 'datetime', 'datetime64']).dtypes
    print('datetime_cols:')
    print(get_column_types(dt_dtpyes))
    g_dtypes = df.drop(dt_dtpyes.index, axis=1).dtypes
    print('other_cols:')
    print(get_column_types(g_dtypes))


def optimize_numeric_values(df):
    optimized_df = df.head(0)
    
    df_int = df.select_dtypes(include='int')
    converted_int = df_int.apply(pd.to_numeric, downcast='integer')
    
    df_float = df.select_dtypes(include='float')
    converted_float = df_float.apply(pd.to_numeric, downcast='float')

    optimized_cols = converted_int.columns.tolist() + converted_float.columns.tolist()
    not_optimized_cols = df.columns[~df.columns.isin(optimized_cols)]
    
    # merge
    optimized_df[df_int.columns] = converted_int
    optimized_df[df_float.columns] = converted_float
    optimized_df[not_optimized_cols] = df[not_optimized_cols]
    
    return optimized_df

# Find the optimized data types

## Action

### optimize numeric columns memory usage: 5.8 GB -> 4.5 GB

In [223]:
%time action = pd.read_csv('{}/All_Action.csv'.format(interim_path))

CPU times: user 49.4 s, sys: 16.2 s, total: 1min 5s
Wall time: 1min 12s


In [224]:
action.info(null_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50601736 entries, 0 to 50601735
Data columns (total 7 columns):
user_id     50601736 non-null float64
sku_id      50601736 non-null int64
time        50601736 non-null object
model_id    29945840 non-null float64
type        50601736 non-null int64
cate        50601736 non-null int64
brand       50601736 non-null int64
dtypes: float64(2), int64(4), object(1)
memory usage: 5.8 GB


In [225]:
for dtype in ['float', 'int', 'object']:
    selected_dtype = action.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype, mean_usage_mb))

Average memory usage for float columns: 257.37 MB
Average memory usage for int columns: 308.85 MB
Average memory usage for object columns: 1833.79 MB


In [ ]:
# handle null values
action.model_id = action.model_id.fillna(-1).astype(int)

In [226]:
action_int = action.select_dtypes(include='int')
%time converted_int = action_int.apply(pd.to_numeric, downcast='signed')

CPU times: user 8.8 s, sys: 8.2 s, total: 17 s
Wall time: 20.7 s


In [227]:
compare_mem_usage(action_int, converted_int)

1544.24 MB -> 386.06 MB
       before  after
int8      NaN    2.0
int16     NaN    1.0
int32     NaN    1.0
int64     4.0    NaN


In [228]:
action_float = action.select_dtypes(include='float')
%time converted_float = action_float.apply(pd.to_numeric, downcast='signed')

CPU times: user 4.44 s, sys: 3.77 s, total: 8.21 s
Wall time: 9.33 s


In [229]:
compare_mem_usage(action_float, converted_float)

1544.24 MB -> 386.06 MB
         before  after
int32       NaN      1
float64     2.0      1


In [230]:
# create new dataframe with the same schema
optimized_action = action.head(0)
optimized_action[converted_int.columns] = converted_int
optimized_action[converted_float.columns] = converted_float

In [231]:
optimized_action['time'] = action.time

In [232]:
compare_mem_usage(action, optimized_action)

1544.24 MB -> 386.06 MB
         before  after
int8        NaN    2.0
int16       NaN    1.0
int32       NaN    2.0
int64       4.0    NaN
float64     2.0    1.0
object      1.0    1.0


In [233]:
optimized_action.info(null_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50601736 entries, 0 to 50601735
Data columns (total 7 columns):
user_id     50601736 non-null int32
sku_id      50601736 non-null int32
time        50601736 non-null object
model_id    29945840 non-null float64
type        50601736 non-null int8
cate        50601736 non-null int8
brand       50601736 non-null int16
dtypes: float64(1), int16(1), int32(2), int8(2), object(1)
memory usage: 4.5 GB


### Optimize Time String (Object) to DateTimeIndex: 4.5 GB -> 1.0 GB

In [234]:
mem_usage(optimized_action.time)

'3667.58 MB'

In [235]:
mem_usage(pd.to_datetime(optimized_action.time, format='%Y-%m-%d %H:%M:%S'))

'386.06 MB'

In [236]:
optimized_action.time = pd.to_datetime(optimized_action.time, format='%Y-%m-%d %H:%M:%S')

In [237]:
dtypes = optimized_action.drop('time', axis=1).dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

In [238]:
column_types

{'user_id': 'int32',
 'sku_id': 'int32',
 'model_id': 'float64',
 'type': 'int8',
 'cate': 'int8',
 'brand': 'int16'}

In [47]:
column_types = {
    'user_id': 'int32',
    'sku_id': 'int32',
    'model_id': 'float64',
    'type': 'int8',
    'cate': 'int8',
    'brand': 'int16'
}

%time optimized_action = pd.read_csv('{}/All_Action.csv'.format(interim_path), dtype=column_types, parse_dates=['time'], infer_datetime_format=True)

CPU times: user 1min 17s, sys: 10.8 s, total: 1min 28s
Wall time: 1min 35s


In [244]:
optimized_action.model_id.fillna(-1, inplace=True)

In [245]:
optimized_action.model_id = pd.to_numeric(optimized_action.model_id.astype(int), downcast='integer')

In [246]:
optimized_action.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50601736 entries, 0 to 50601735
Data columns (total 7 columns):
user_id     int32
sku_id      int32
time        datetime64[ns]
model_id    int16
type        int8
cate        int8
brand       int16
dtypes: datetime64[ns](1), int16(2), int32(2), int8(2)
memory usage: 1.0 GB


In [247]:
%time optimized_action.to_csv('{}/All_Action.csv'.format(interim_path), index=False)

CPU times: user 8min 49s, sys: 8.65 s, total: 8min 58s
Wall time: 9min 31s


## Action - Remove duplicated rows

In [5]:
%time action = pd.read_csv('{}/All_Action.csv'.format(interim_path), parse_dates=['time'], infer_datetime_format=True)

CPU times: user 1min 23s, sys: 19.3 s, total: 1min 42s
Wall time: 1min 58s


In [6]:
action.head()

,user_id,sku_id,time,model_id,type,cate,brand
0,266079,138778,2016-01-31 23:59:02,-1,1,8,403
1,266079,138778,2016-01-31 23:59:03,0,6,8,403
2,200719,61226,2016-01-31 23:59:07,-1,1,8,30
3,200719,61226,2016-01-31 23:59:08,0,6,8,30
4,263587,72348,2016-01-31 23:59:08,-1,1,5,159


In [ ]:
action.info()

In [ ]:
action.model_id = action.model_id.fillna(-1).astype(int)

In [8]:
action.isnull().sum()

user_id     0
sku_id      0
time        0
model_id    0
type        0
cate        0
brand       0
dtype: int64

In [9]:
optimized_action = optimize_numeric_values(action)

In [15]:
optimized_action.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50601736 entries, 0 to 50601735
Data columns (total 7 columns):
user_id     int32
sku_id      int32
time        datetime64[ns]
model_id    int16
type        int8
cate        int8
brand       int16
dtypes: datetime64[ns](1), int16(2), int32(2), int8(2)
memory usage: 1.0 GB


In [10]:
show_dtypes(optimized_action)

datetime_cols:
{'time': 'datetime64[ns]'}
other_cols:
{'user_id': 'int32', 'sku_id': 'int32', 'model_id': 'int16', 'type': 'int8', 'cate': 'int8', 'brand': 'int16'}


In [69]:
column_types = {
    'user_id': 'int32',
    'sku_id': 'int32',
    'model_id': 'int16',
    'type': 'int8',
    'cate': 'int8',
    'brand': 'int16'
}

%time optimized_action = pd.read_csv('{}/All_Action.csv'.format(interim_path), dtype=column_types, parse_dates=['time'], infer_datetime_format=True)

CPU times: user 1min 20s, sys: 10.6 s, total: 1min 31s
Wall time: 1min 43s


In [12]:
# drop duplicated rows
optimized_action.duplicated().sum()

13513841

In [ ]:
optimized_action.head(10000)[optimized_action.head(10000).duplicated()].head(20)

In [13]:
%time dedup_optimized_action = optimized_action[~optimized_action.duplicated()]

CPU times: user 32.6 s, sys: 27.6 s, total: 1min
Wall time: 1min 19s


In [14]:
dedup_optimized_action.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37087895 entries, 0 to 50601735
Data columns (total 7 columns):
user_id     int32
sku_id      int32
time        datetime64[ns]
model_id    int16
type        int8
cate        int8
brand       int16
dtypes: datetime64[ns](1), int16(2), int32(2), int8(2)
memory usage: 1.0 GB


In [16]:
del optimized_action
import gc
gc.collect()

21

In [18]:
%time dedup_optimized_action.to_csv('{}/Dedup_All_Action.csv'.format(interim_path), index=False)

CPU times: user 6min 42s, sys: 8.42 s, total: 6min 50s
Wall time: 7min 9s


## User

In [74]:
%time user = pd.read_csv('{}/JData_User.csv'.format(raw_path), encoding='gbk', parse_dates=['user_reg_tm'], infer_datetime_format=True)

CPU times: user 168 ms, sys: 25.1 ms, total: 193 ms
Wall time: 208 ms


In [75]:
user.head()

,user_id,age,sex,user_lv_cd,user_reg_tm
0,200001,56岁以上,2.0,5,2016-01-26
1,200002,-1,0.0,1,2016-01-26
2,200003,36-45岁,1.0,4,2016-01-26
3,200004,-1,2.0,1,2016-01-26
4,200005,16-25岁,0.0,4,2016-01-26


In [76]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105321 entries, 0 to 105320
Data columns (total 5 columns):
user_id        105321 non-null int64
age            105318 non-null object
sex            105318 non-null float64
user_lv_cd     105321 non-null int64
user_reg_tm    105318 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 4.0+ MB


In [77]:
# convert age column dtype to integer
user_age_mapping = {
    '15岁以下': 0,
    '16-25岁': 1,
    '26-35岁': 2,
    '36-45岁': 3,
    '46-55岁': 4,
    '56岁以上': 5,
}

user['age'] = user.age.map(user_age_mapping).fillna(-1).astype(int)

In [78]:
user.isnull().sum()

user_id        0
age            0
sex            3
user_lv_cd     0
user_reg_tm    3
dtype: int64

In [79]:
user[user.isnull().any(axis=1)]

,user_id,age,sex,user_lv_cd,user_reg_tm
34072,234073,-1,NaN,1,NaT
38905,238906,-1,NaN,1,NaT
67704,267705,-1,NaN,1,NaT


In [80]:
# sex = 2 means secret
user.sex = user.sex.fillna(2).astype(int)

In [81]:
user.user_reg_tm.fillna(method='ffill', inplace=True)

In [82]:
user.isnull().sum()

user_id        0
age            0
sex            0
user_lv_cd     0
user_reg_tm    0
dtype: int64

In [83]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105321 entries, 0 to 105320
Data columns (total 5 columns):
user_id        105321 non-null int64
age            105321 non-null int64
sex            105321 non-null int64
user_lv_cd     105321 non-null int64
user_reg_tm    105321 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(4)
memory usage: 4.0 MB


In [84]:
optimized_user = user.head(0)

In [110]:
optimized_user = optimize_numeric_values(user)

In [177]:
show_dtypes(optimized_user)

datetime_cols:
{'user_reg_tm': 'datetime64[ns]'}
other_cols:
{'user_id': 'int32', 'age': 'int8', 'sex': 'int8', 'user_lv_cd': 'int8'}


In [167]:
optimized_user.to_csv('{}/user.csv'.format(interim_path), index=False)

In [55]:
column_types = {'user_id': 'int32', 'age': 'int8', 'sex': 'int8', 'user_lv_cd': 'int8'}
optimized_user = pd.read_csv('{}/user.csv'.format(interim_path), dtype=column_types, parse_dates=['user_reg_tm'], infer_datetime_format=True)

In [56]:
optimized_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105321 entries, 0 to 105320
Data columns (total 5 columns):
user_id        105321 non-null int32
age            105321 non-null int8
sex            105321 non-null int8
user_lv_cd     105321 non-null int8
user_reg_tm    105321 non-null datetime64[ns]
dtypes: datetime64[ns](1), int32(1), int8(3)
memory usage: 1.5 MB


## product

In [199]:
%time product = pd.read_csv('{}/JData_Product.csv'.format(raw_path))

CPU times: user 27.3 ms, sys: 16.2 ms, total: 43.6 ms
Wall time: 44.8 ms


In [200]:
show_nan(product)

missing value columns:
sku_id    0
a1        0
a2        0
a3        0
cate      0
brand     0
dtype: int64


In [201]:
product.info(null_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24187 entries, 0 to 24186
Data columns (total 6 columns):
sku_id    24187 non-null int64
a1        24187 non-null int64
a2        24187 non-null int64
a3        24187 non-null int64
cate      24187 non-null int64
brand     24187 non-null int64
dtypes: int64(6)
memory usage: 1.1 MB


In [202]:
optimized_product = optimize_numeric_values(product)

In [203]:
optimized_product.info(null_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24187 entries, 0 to 24186
Data columns (total 6 columns):
sku_id    24187 non-null int32
a1        24187 non-null int8
a2        24187 non-null int8
a3        24187 non-null int8
cate      24187 non-null int8
brand     24187 non-null int16
dtypes: int16(1), int32(1), int8(4)
memory usage: 236.3 KB


In [204]:
show_dtypes(optimized_product)

datetime_cols:
{}
other_cols:
{'sku_id': 'int32', 'a1': 'int8', 'a2': 'int8', 'a3': 'int8', 'cate': 'int8', 'brand': 'int16'}


In [ ]:
column_types = {'sku_id': 'int32', 'a1': 'int8', 'a2': 'int8', 'a3': 'int8', 'cate': 'int8', 'brand': 'int16'}
optimized_product = pd.read_csv('{}/JData_Product.csv'.format(raw_path), dtype=column_types)

## comment

In [205]:
%time comment = pd.read_csv('{}/JData_Comment.csv'.format(raw_path), parse_dates=['dt'], infer_datetime_format=True)

CPU times: user 676 ms, sys: 77 ms, total: 754 ms
Wall time: 807 ms


In [206]:
comment.info(null_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558552 entries, 0 to 558551
Data columns (total 5 columns):
dt                  558552 non-null datetime64[ns]
sku_id              558552 non-null int64
comment_num         558552 non-null int64
has_bad_comment     558552 non-null int64
bad_comment_rate    558552 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 21.3 MB


In [207]:
show_nan(comment)

missing value columns:
dt                  0
sku_id              0
comment_num         0
has_bad_comment     0
bad_comment_rate    0
dtype: int64


In [218]:
comment[['dt', 'sku_id']].duplicated().sum()

0

In [208]:
optimized_comment = optimize_numeric_values(comment)

In [209]:
optimized_comment.info(null_counts=True, memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 558552 entries, 0 to 558551
Data columns (total 5 columns):
dt                  558552 non-null datetime64[ns]
sku_id              558552 non-null int32
comment_num         558552 non-null int8
has_bad_comment     558552 non-null int8
bad_comment_rate    558552 non-null float32
dtypes: datetime64[ns](1), float32(1), int32(1), int8(2)
memory usage: 9.6 MB


In [210]:
show_dtypes(optimized_comment)

datetime_cols:
{'dt': 'datetime64[ns]'}
other_cols:
{'sku_id': 'int32', 'comment_num': 'int8', 'has_bad_comment': 'int8', 'bad_comment_rate': 'float32'}


In [26]:
column_types = {'sku_id': 'int32', 'comment_num': 'int8', 'has_bad_comment': 'int8', 'bad_comment_rate': 'float32'}
%time optimized_comment = pd.read_csv('{}/JData_Comment.csv'.format(raw_path), dtype=column_types, parse_dates=['dt'], infer_datetime_format=True)

CPU times: user 837 ms, sys: 74.1 ms, total: 911 ms
Wall time: 1.1 s


In [27]:
optimized_comment.head()

,dt,sku_id,comment_num,has_bad_comment,bad_comment_rate
0,2016-02-01,1000,3,1,0.0417
1,2016-02-01,10000,2,0,0.0000
2,2016-02-01,100011,4,1,0.0376
3,2016-02-01,100018,3,0,0.0000
4,2016-02-01,100020,3,0,0.0000


# Value Inconsistency Check

### sku_id in action > sku_id in product? yes

In [ ]:
import gc
column_types = {
    'user_id': 'int32',
    'sku_id': 'int32',
    'model_id': 'int16',
    'type': 'int8',
    'cate': 'int8',
    'brand': 'int16'
}

%time optimized_action = pd.read_csv('{}/All_Action.csv'.format(interim_path), dtype=column_types, parse_dates=['time'], infer_datetime_format=True)

print(optimized_action.sku_id.nunique())
del optimized_action
gc.collect()

%time dedup_optimized_action.to_csv('{}/Dedup_All_Action.csv'.format(interim_path), index=False)
print(dedup_optimized_action.sku_id.nunique())
del optimized_action
gc.collect()

column_types = {'sku_id': 'int32', 'a1': 'int8', 'a2': 'int8', 'a3': 'int8', 'cate': 'int8', 'brand': 'int16'}
optimized_product = pd.read_csv('{}/JData_Product.csv'.format(raw_path), dtype=column_types)
print(optimized_product.sku_id.nunique())
del optimized_product
gc.collect()

# 28710
# 28710
# 24187

### sku_id has only one pair (cate, brand)? yes

In [ ]:
dedup_optimized_action = pd.read_csv('{}/Dedup_All_Action.csv'.format(interim_path))
g = dedup_optimized_action[['sku_id', 'cate', 'brand']].drop_duplicates().groupby('sku_id')
print(g.brand.count().value_counts())
print(g.cate.count().value_counts())

del dedup_optimized_action, g
gc.collect()

In [72]:
column_types = {'sku_id': 'int32', 'a1': 'int8', 'a2': 'int8', 'a3': 'int8', 'cate': 'int8', 'brand': 'int16'}
optimized_product = pd.read_csv('{}/JData_Product.csv'.format(raw_path), dtype=column_types)

In [74]:
c = optimized_product.groupby('sku_id').count()

In [75]:
c.cate.value_counts()

1    24187
Name: cate, dtype: int64

In [76]:
c.brand.value_counts()

1    24187
Name: brand, dtype: int64